# 1. Getting Started with Tensorflow

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 03/09/2024   | Martin | Created   | Started chapter 1 | 
| 09/09/2024   | Martin | Update   | To page 55 - activation functions | 

# Content

* [Introduction](#introduction)
* [Variables and Tensors](#variables-and-tensors)
* [Activation Functions](#activation-functions)
* [Tensorflow Under the hood](#tensorflow-under-the-hood)

In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

# Introduction

General workflow of Tensorflow:

1. Import or generate datasets
2. Transform and normalize data
3. Partition datasets into training, test and validation sets
4. Set algorithm parameters (hyperparameters)
5. Initialize variables
6. Define the model structure
7. Declare loss function
8. Initialise and train the model
9. Evaluate the model
10. Tune hyper parameters
11. Deploy/ predict new outcomes

In [13]:
# 1.
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

data = tfds.load("iris", split='train')

# 4.
epochs = 1000
batch_size = 32
input_size = 4
output_size = 3
learning_rate = 0.001

# 5.
weights = tf.Variable(tf.random.normal(
  shape=(input_size, output_size),
  dtype=tf.float32
))
biases = tf.Variable(tf.random.normal(
  shape=(output_size,),
  dtype=tf.float32
))

# 8.
optimizer = tf.optimizers.SGD(learning_rate)

for _ in range(epochs):
  # 2.
  for batch in data.batch(batch_size, drop_remainder=True):
    labels = tf.one_hot(batch['label'], 3)
    X = batch['features']
    X = (X - np.mean(X) / np.std(X))

    with tf.GradientTape() as tape:
      # 6.
      logits = tf.add(tf.matmul(X, weights), biases) # logistic regression (actually is linear regression first)

      # 7.
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels, logits)
      )

    # 8.
    gradients = tape.gradient(loss, [weights, biases])
    optimizer.apply_gradients(zip(gradients, [weights, biases]))

print(f"final loss is: {loss.numpy():.3f}")
preds = tf.math.argmax(tf.add(tf.matmul(X, weights), biases), axis=1)
ground_truth = tf.math.argmax(labels, axis=1)
for y_true, y_pred in zip(ground_truth.numpy(), preds.numpy()):
  print(f"real label: {y_true} | fitted: {y_pred}")

final loss is: 0.313
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 1 | fitted: 2
real label: 2 | fitted: 1
real label: 2 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 1 | fitted: 1
real label: 2 | fitted: 2
real label: 1 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 1 | fitted: 1
real label: 0 | fitted: 0
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2


Tensorflow computes the changes by creating a computational graph which tracks the steps taken for all operations. Graphs do not use recursion.

Tensorflow keeps track of all variables in the computational graph and computes the gradients to minimize the loss

# Variables and Tensors

All variables are stored as tensors. Even single number/ digits are stored as zero-dimensional tensors

In [1]:
import tensorflow as tf

2024-09-09 14:51:18.603813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-09 14:51:18.739735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 14:51:18.802387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 14:51:18.817166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 14:51:18.916843: I tensorflow/core/platform/cpu_feature_guar

In [5]:
row_dim, col_dim = 3, 3
ones_tsr = tf.ones([row_dim, col_dim])
ones_tsr

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [6]:
# or a filled tensor
tf.fill([row_dim, col_dim], value=42)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[42, 42, 42],
       [42, 42, 42],
       [42, 42, 42]], dtype=int32)>

In [8]:
# create tensors based on an existing shape
tf.zeros_like(ones_tsr)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [9]:
# generate values from distributions
tf.random.uniform([row_dim, col_dim], minval=0, maxval=1)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.9851533 , 0.11942112, 0.5387734 ],
       [0.4384067 , 0.9586129 , 0.19684386],
       [0.04166341, 0.10652184, 0.03117621]], dtype=float32)>

To initialize a tensor as a variable use `tf.Variable()`. Notice in the example below that the output is a Variable instead of tensor now

In [11]:
my_var = tf.Variable(tf.zeros([row_dim, col_dim]))
my_var

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [14]:
import numpy as np
# to convert into a tensor
np_arr = np.array([1, 2, 3])
l = [1, 2, 3]
np_arr, tf.convert_to_tensor(np_arr)

(array([1, 2, 3]),
 <tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 3])>)

## Creating Matrices

pg 52-54 examples of elementwise operations on matrices

Any custom functions created must use the tensorflow API to be used in the computational graph

In [15]:
id_matrix = tf.linalg.diag([1.0, 1.0, 1.0])
A = tf.random.truncated_normal([2, 3])
B = tf.fill([2, 3], 5.0)
C = tf.random.uniform([3, 2])
D = tf.convert_to_tensor(np.array([[1., 2., 3.],
                                   [-3., -7., -1.],
                                   [0., 5., -2.]]),
                                   dtype=tf.float32)

In [16]:
# Operations
A + B

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[6.7755075, 5.012356 , 5.9779267],
       [4.1081805, 4.056792 , 5.404626 ]], dtype=float32)>

In [19]:
tf.matmul(B, id_matrix)
tf.multiply(D, id_matrix)
tf.transpose(D)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 1., -3.,  0.],
       [ 2., -7.,  5.],
       [ 3., -1., -2.]], dtype=float32)>

In [22]:
# Linear algebra uses the .linalg methods
tf.linalg.det(D)
tf.linalg.inv(D)
tf.linalg.eigh(D)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-10.659076  ,  -0.22750677,   2.8865824 ], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[ 0.21749546, -0.6325011 ,  0.7433963 ],
        [ 0.84526515, -0.25879973, -0.46749282],
        [-0.48808047, -0.7300446 , -0.47834337]], dtype=float32)>)

# Activation Functions 

In [ ]:
# pg 55

# Tensorflow Under the Hood

1. Tensorflow uses eager execution